In [34]:
import numpy as np 
import pandas as pd 

from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit #for data preprocessing and crass validating 
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import HistGradientBoostingRegressor

from statistics import mean
from hyperopt import Trials, hp, fmin, tpe, STATUS_OK, space_eval #for hyperparameter tuning and minimizing

from datetime import date
from datetime import datetime
from datetime import timedelta

import tensorflow as tf
import keras
import keras.layers as layers
from keras.layers import Dense
from keras.models import Sequential
from keras.utils import to_categorical
from keras.optimizers import SGD 
from keras.callbacks import EarlyStopping
from keras.losses import MeanSquaredError
from keras.layers import LSTM

import itertools

from sklearn.discriminant_analysis import StandardScaler


import seaborn as sns 

from scipy.stats import boxcox 
from scipy.special import inv_boxcox

from termcolor import colored

import joblib
from tqdm import tqdm 

import swifter
from pandarallel import pandarallel

In [35]:
tf.__version__
model = tf.keras.models.load_model('model2.keras')

In [36]:
samples = pd.read_csv('preprocessed_lstm.csv')
building_id = pd.read_csv('devices.csv')
test = pd.read_csv('test.csv', header = None)
reading_types = pd.read_csv('reading_types.csv')

samples['date'] = pd.to_datetime(samples['date'])

predictions = test
value_type_ids = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12']

train = pd.DataFrame()

In [37]:
for building, df in samples.groupby('building_id'): 
    df = df.set_index('date', drop = False) 
    idx = pd.date_range('01-01-2023', '12-31-2023 23:55', freq = '5min')
    df = df.reindex(idx, fill_value = building)
    df.index = pd.to_datetime(df.index)
    df['date'] = df.index

    df = df.drop(['day type', 'Winter', 'Spring', 'Summer', 'Fall', 'work_hours', 'trimester_day'], axis = 1)
    
    df['day type'] = df['date'].dt.dayofweek.map({
        0: 1,
        1: 1,
        2: 1,
        3: 1,
        4: 1,
        5: 0, 
        6: 0
    })

        
    df['season'] = df['date'].dt.month.map({
        1: 'Winter',
        2: 'Winter',
        3: 'Spring',
        4: 'Spring',
        5: 'Spring',
        6: 'Summer',
        7: 'Summer',
        8: 'Summer',
        9: 'Fall',
        10: 'Fall',
        11: 'Fall',
        12: 'Winter'
    })

    df['work_hours'] = df['date'].dt.hour.between(8, 18)
    df['work_hours'].map({True: 1, False: 0})

    df['Winter'] = df['season'].map({
        'Winter': 1,
        'Spring': 0,
        'Summer': 0,
        'Fall': 0
    })
    df['Spring'] = df['season'].map({
        'Winter': 0,
        'Spring': 1,
        'Summer': 0,
        'Fall': 0
    })
    df['Summer'] = df['season'].map({
        'Winter': 0,
        'Spring': 0,
        'Summer': 1,
        'Fall': 0
    })
    df['Fall'] = df['season'].map({
        'Winter': 0,
        'Spring': 0,
        'Summer': 0,
        'Fall': 1
    })


    df = df.drop('season', axis = 1)

    df[value_type_ids] = df[value_type_ids].interpolate(method = 'linear').bfill().ffill()

    df.info()

    train = pd.concat([train, df])

train.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 105120 entries, 2023-01-01 00:00:00 to 2023-12-31 23:55:00
Freq: 5min
Data columns (total 21 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Unnamed: 0   105120 non-null  int64         
 1   date         105120 non-null  datetime64[ns]
 2   building_id  105120 non-null  int64         
 3   1            105120 non-null  float64       
 4   2            105120 non-null  float64       
 5   3            105120 non-null  float64       
 6   4            105120 non-null  float64       
 7   5            105120 non-null  float64       
 8   6            105120 non-null  float64       
 9   7            105120 non-null  float64       
 10  8            105120 non-null  float64       
 11  9            105120 non-null  float64       
 12  10           105120 non-null  float64       
 13  11           105120 non-null  float64       
 14  12           105120 non-null  float64  

In [38]:
scaler = StandardScaler()  
scaler = scaler.fit(samples[value_type_ids]) 

train[value_type_ids] = scaler.transform(train[value_type_ids])


In [39]:
predictions.columns = ['device_id', 'date', 'value_type_id']
predictions = pd.merge(predictions, building_id, on='device_id', how='inner')
predictions['date'] =  pd.to_datetime(predictions['date'])
predictions['floored_date'] = predictions['date'].dt.floor('5min')

In [40]:
building_encoder = pd.get_dummies(predictions['building_id'])
predictions = predictions.join(building_encoder.add_suffix('_b'))

predictions['day type'] = predictions['date'].dt.dayofweek.map({
    0: 1,
    1: 1,
    2: 1,
    3: 1,
    4: 1,
    5: 0, 
    6: 0
})

predictions['work_hours'] = predictions['date'].dt.hour.between(8, 18)
predictions['work_hours'].map({True: 1, False: 0})

predictions['season'] = predictions['date'].dt.month.map({
    1: 'Winter',
    2: 'Winter',
    3: 'Spring',
    4: 'Spring',
    5: 'Spring',
    6: 'Summer',
    7: 'Summer',
    8: 'Summer',
    9: 'Fall',
    10: 'Fall',
    11: 'Fall',
    12: 'Winter'
})

season_encoder = pd.get_dummies(predictions['season'])
predictions = predictions.join(season_encoder)
predictions = predictions.drop('season', axis = 1)


In [41]:
predictions.info() 
progressBar = tqdm(total = len(predictions))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7856720 entries, 0 to 7856719
Data columns (total 40 columns):
 #   Column         Dtype         
---  ------         -----         
 0   device_id      int64         
 1   date           datetime64[ns]
 2   value_type_id  int64         
 3   building_id    int64         
 4   floored_date   datetime64[ns]
 5   1_b            bool          
 6   2_b            bool          
 7   3_b            bool          
 8   6_b            bool          
 9   8_b            bool          
 10  10_b           bool          
 11  11_b           bool          
 12  12_b           bool          
 13  13_b           bool          
 14  16_b           bool          
 15  17_b           bool          
 16  18_b           bool          
 17  19_b           bool          
 18  20_b           bool          
 19  21_b           bool          
 20  23_b           bool          
 21  24_b           bool          
 22  25_b           bool          
 23  26_b   

  0%|          | 1496/7856720 [1:22:33<7225:33:57,  3.31s/it]


In [49]:
def get_predictions(row):
    type = row['value_type_id']
    date = row['floored_date'] 
    building = row['building_id']

    classes = row.drop(['device_id', 'date',  'value_type_id', 'floored_date', 'building_id', 'value'])

    start_date = date - timedelta(minutes=50)
    end_date = date - timedelta(minutes=5)
    inputs = train[(train['building_id'] == building)]
    inputs = inputs.loc[(inputs['date'] >= start_date) & (inputs['date'] <= end_date)]

    inputs = inputs[value_type_ids]

    inputs = inputs.astype('float32')
    classes = classes.astype('float32')

    inputs = scaler.transform(inputs)
    print(inputs)
    print(classes)

    dataX = []
    class_X = []

    dataX.append(inputs[0:10, :])  # Use iloc for DataFrame slicing
    class_X.append(classes) 

    dataX = np.array(dataX) 
    class_X = np.array(class_X)


    prediction = model.predict({
        "values": dataX,
        "class": class_X
    }, verbose = 0)


    pred = scaler.inverse_transform(prediction)
    

    progressBar.update(1)
    print(pred)

    return pred[0][type - 1] #prob have to grab the value or smt





In [50]:
predictions = predictions.head(1)
predictions['value'] = predictions.apply(get_predictions, axis=1)

[[ -5.975783    -1.2940383   -1.1326131    0.          -9.844728
  -15.605975    -1.732767    -0.46325067 -10.547973    -0.9482282
   -9.675414    -3.4417214 ]
 [ -5.9753933   -1.2940383   -1.1329565    0.          -9.844728
  -15.605975    -1.731103    -0.46325067 -10.547973    -0.9477804
   -9.626514    -3.4517977 ]
 [ -5.975134    -1.2940383   -1.1326588    0.          -9.844728
  -15.605975    -1.7377594   -0.46325067 -10.598565    -0.9475564
   -9.577615    -3.4618735 ]
 [ -5.9773407   -1.4502418   -1.132865     0.          -9.844728
  -15.605975    -1.734431    -0.46325067 -10.547973    -0.9468847
   -9.528715    -3.4719498 ]
 [ -5.9770813   -1.4502418   -1.1331246    0.          -9.844728
  -15.605975    -1.7394233   -0.46325067 -10.547973    -0.9468847
   -9.4798155   -3.482026  ]
 [ -5.9751596   -1.4502418   -1.1335826    0.          -9.844728
  -15.605975    -1.7419194   -0.46325067 -10.547973    -0.9464369
   -9.430916    -3.4921021 ]
 [ -5.974809    -1.4502418   -1.1334718 

InvalidArgumentError: Graph execution error:

Detected at node model_9/dense_9/Relu defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "C:\Users\jonat\AppData\Roaming\Python\Python311\site-packages\ipykernel_launcher.py", line 18, in <module>

  File "C:\Users\jonat\AppData\Roaming\Python\Python311\site-packages\traitlets\config\application.py", line 1075, in launch_instance

  File "C:\Users\jonat\AppData\Roaming\Python\Python311\site-packages\ipykernel\kernelapp.py", line 739, in start

  File "C:\Users\jonat\AppData\Roaming\Python\Python311\site-packages\tornado\platform\asyncio.py", line 205, in start

  File "c:\Users\jonat\AppData\Local\Programs\Python\Python311\Lib\asyncio\base_events.py", line 608, in run_forever

  File "c:\Users\jonat\AppData\Local\Programs\Python\Python311\Lib\asyncio\base_events.py", line 1936, in _run_once

  File "c:\Users\jonat\AppData\Local\Programs\Python

In [ ]:
predictions = predictions.drop(['floored_date', 'building_id'], axis = 1)
predictions.to_csv('predictions.csv', header = False)